# Input Output
# Modules

In [48]:
:option no-lint
:option no-show-types
import Data.List

## Finally .. Hello World


In [49]:
main = putStrLn "Hello, World!"
main

Hello, World!

Compile and run from the command line:
```shell
❯ stack ghc hello.hs
[1 of 2] Compiling Main             ( hello.hs, hello.o )
[2 of 2] Linking hello
❯ ./hello
Hello, World!
```

## Types in IO operations

- ALL IO operations has some type `IO a`
- Where `IO` is a `data` type: `data IO a`
- For Example:
    - `getLine::IO String`
    - `putStrLn:String -> IO ()`
    - main can be any `IO a`
- Remember `()` is the tuple with no elements: **Unit**


## Another example

Printing a value of any type that can be converted to a `String`

In [50]:
print::(Show a) => a -> IO ()
print x = putStrLn (show x)

main:: IO ()
main = print [1, 2, 3]

main

[1,2,3]

## How to make several IO actions ?


The `do` notation.
For example:

In [51]:
main = do
            putStrLn "Welcome!"
            putStrLn "Bienvenido!"
            putStrLn "Bienvenue!"
            putStrLn "Benvenuto!"

main

Welcome!
Bienvenido!
Bienvenue!
Benvenuto!

## How to use the value `returned` from an `IO` operation ?

For example I want to read a line from the input (`getLine`) and then print it

In [52]:
main = putStrLn ("Hello " ++ getLine) 

: 

Houston, we've had a problem here

## Do syntax extracting values from an `IO` operation 

In [53]:
main = do
         name <- getLine
         putStrLn ("Hello " ++ name) 

Compile and run from the command line:
```shell
❯ stack ghc hello1
[1 of 2] Compiling Main             ( hello1.hs, hello1.o )
[2 of 2] Linking hello1
❯ ./hello1
John
Hello John
```

### Writting a function that returns an `IO a`

For example I want to write a function that reads a line and returns it reversed

```haskell
readReversed = do
                 name <- getLine
                 reverse name
```                 
It doesn't work, because `reverse` returns a `String` and haskell is expecting an `IO` of some type (Why??)

You can use the `return` function: `return::x -> IO x`. It takes any value and _wraps_ it into an `IO`
```haskell
readReversed::IO String
readReversed = do
                 name <- getLine
                 return (reverse name)
```                 

## Create a loop using an if

In [54]:
-- We can do:
printN :: (Show a) => Int -> a -> IO ()
printN n x = do if n <= 0 then return ()
                else do
                    print x
                    printN (n-1) x
printN 3 "Hello!"

"Hello!"
"Hello!"
"Hello!"

Actually there is a function `unless` defined like:
```haskell
unless::Bool -> IO () -> IO ()
unless b e = do
    if b then return () else e
```    

```haskell
printN n x = unless (n <= 0) $ do
                print x
                printN (n-1) x
             

```

## Number lines of a file

In [55]:
-- Read the input until eof and print the lines numbered
import System.IO( isEOF )

main = num 0

num:: Int -> IO ()
num n = do
           eof <- isEOF
           unless eof $ do
                line <- getLine
                putStrLn $ show n ++ ": " ++ line
                num (n + 1)             

## Read from a disk file

In [56]:
numFile::String -> IO ()
numFile fileName = do
   content <- readFile fileName
   printLines 1 (lines content)

printLines::Int -> [String] -> IO ()
printLines _ [] = return ()
printLines n (x:xs) = do
                         putStrLn $ show n ++ ": "++ x
                         printLines (n+1) xs
                         
numFile "text/lorem-ipsum.txt"  


1: Lorem ipsum dolor sit amet, consectetur adipiscing elit.
2: Donec magna odio, placerat nec nulla in, tempus rhoncus lorem.
3: Curabitur vitae dignissim erat. Suspendisse egestas magna et tempus rutrum.

## Using, command line arguments

In [57]:
import System.Environment

main = do
   args <- getArgs
   if length args == 0 then do
       putStrLn "Please specify the filename"
       return ()
   else 
       numFile (head args) 
       

## Modules


* A `module` defines a collection of definitions: **functions**, **types (data & others)**, **classes** and **values**

* In an environment created by a set of `imports` (resources brought into scope from other modules). 

* It `exports` some of these resources, making them available to other modules 

* A Haskell **program** is a collection of modules, one of which, by convention, must be called `Main` and must export the value `main`, which must be a computation of type `IO t` for some type `t` 

* A `package` is a **library** of Haskell `modules`.
    - There is a default [base](https://hackage.haskell.org/package/base) package that contains (btween others) the, imported by default, `Prelude` module, plus many others.
    - Many other packages can be obtained from [The haskell package Repository](https://hackage.haskell.org)
    - And of course you can build your own

   * Tools like [Stack](https://docs.haskellstack.org/en/stable/README/) simplify project building, including installation of needed packages. 

## Module Example

```haskell
module WordCount (wordCount, wordsOf) where

import Data.Char
import qualified Data.Map as M

wordCount :: String -> Result
wordCount text = foldl addToMap M.empty (wordsOf text)
  where addToMap m w = M.insertWith (+) w 1 m

normalize:: String -> String
normalize (x:xs) = toLower x : normalize xs

-- etc
```

* Here we are implementing a `module` **WordCount**, that defines several functions, but **only** exports the functions **wordCount** and **wordsOf**.

* We are importing the `Data.Char` module and the `Data.Map` one.

* Data.Map is imported qualified as `M`, so we have to prefix its references with `M.` as in `M.insertWith`

## Import Syntax Alternatives

```haskell
module A(x,y) where
```

| Import declaration              |	Names brought into scope  |
|---------------------------------|---------------------------|
| import A	                      | x, y, A.x, A.y            |
| import A(x)	                  | x, A.x                    |
| import qualified A              |	A.x, A.y                  |
| import qualified A as B	      | B.x, B.y                  |
| import A hiding (x)	          | y, A.y                    |

| Import declaration              |	Names brought into scope  |
|---------------------------------|---------------------------|
| import A()	                  | (nothing)                 |
| import qualified A()	          | (nothing)                 |
| import qualified A(x)	          | A.x                       |
| import A hiding ()	          | x, y, A.x, A.y            |
| import qualified A hiding ()    | A.x, A.y                  |
| import qualified A hiding (x)	  | A.y                       |
| import A as B	                  | x, y, B.x, B.y            |
| import A as B(x)	              | x, B.x                    |

## Exporting

```haskell
module Tree where

data Tree a = Nil | Node a (Tree a) (Tree a) 

singleton x         = Node x Nil Nil
left (Node _ l _)   = l
right(Node _ _ r)   = r
fromList .....
... etc
```

```haskell
module Tree where
```
Exports everything

```haskell
module Tree (Tree, singleton, fromList, left, right) where
```
Export the functions `singleton` ... `right` and the `Tree` type. But **not** the `Nil` and `Node` constructors

```haskell
module Tree (Tree(Nil), ...) where
```
Exports the `Tree` type including the `Nil` constructor.

```haskell
module Tree (Tree(..), ...) where
```
Exports the `Tree` type with **all** its constructors.

## The Map type

> Maps (a.k.a. dictionaries) allow you to store associations between unique keys and values. 

`data Map k v`

`data IntMap v` (Key is an Int)

[See Documentation](https://haskell-containers.readthedocs.io/en/latest/map.html)

Maps are **inmutable** so any update functions do not modify the map that you passed in, they return a new `Map`

In [58]:
import qualified Data.Map as Map

fruits     = Map.fromList[("Apples", 100), ("Oranges", 10)]
moreFruits = Map.insert "Bananas" 1000 fruits

fruits
moreFruits 

fromList [("Apples",100),("Oranges",10)]

fromList [("Apples",100),("Bananas",1000),("Oranges",10)]

## Examples

The number of Apples and Bananas

In [59]:
Map.lookup "Apples" fruits
Map.lookup "Bananas" fruits
Map.findWithDefault 0 "Bananas" fruits
Map.findWithDefault 0 "Bananas" moreFruits

Just 100

Nothing

0

1000

Do I have "Bananas" in the moreFruits Map ?

In [ ]:
Map.member "Bananas" moreFruits

True

Update the number of "Bananas" and delete the Oranges

In [ ]:
moreFruits
fruitSalad = Map.delete "Oranges" $ Map.insert "Bananas" 200 moreFruits
fruitSalad

fromList [("Apples",100),("Bananas",1000),("Oranges",10)]

fromList [("Apples",100),("Bananas",200)]

Create a richer Fruit Salad

In [ ]:
Map.union fruitSalad $ Map.fromList [("Pears", 20), ("Kiwis", 30)]

fromList [("Apples",100),("Bananas",200),("Kiwis",30),("Pears",20)]